In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

%matplotlib inline

In [2]:
from datasets import load_dataset

# data_en = load_dataset('head_qa', 'en')
data_es = load_dataset('head_qa', 'es' )
data_es

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


DatasetDict({
    train: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2657
    })
    test: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 2742
    })
    validation: Dataset({
        features: ['name', 'year', 'category', 'qid', 'qtext', 'ra', 'image', 'answers'],
        num_rows: 1366
    })
})

In [3]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [5]:
for d in training:
    print(d)

{'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'}, {'aid': 2, 'atext': 'Son hiperpolarizantes.'}, {'aid': 3, 'atext': 'Se pueden sumar.'}, {'aid': 4, 'atext': 'Se propagan a largas distancias.'}, {'aid': 5, 'atext': 'Presentan un periodo refractario.'}], 'category': 'biology', 'image': '', 'name': 'Cuaderno_2013_1_B', 'qid': 1, 'qtext': 'Los potenciales postsinápticos excitadores:', 'ra': 3, 'year': '2013'}
{'answers': [{'aid': 1, 'atext': 'Músculo liso.'}, {'aid': 2, 'atext': 'Músculo esquelético.'}, {'aid': 3, 'atext': 'Músculo cardiaco.'}, {'aid': 4, 'atext': 'Huso muscular.'}, {'aid': 5, 'atext': 'Tendón.'}], 'category': 'biology', 'image': '', 'name': 'Cuaderno_2013_1_B', 'qid': 2, 'qtext': 'Placa motora es la unión entre la neurona motora y el:', 'ra': 2, 'year': '2013'}
{'answers': [{'aid': 1, 'atext': 'Fibras musculares lisas.'}, {'aid': 2, 'atext': 'Neuronas bipolares de la retina.'}, {'aid': 3, 'atext': 'Fibras musculares estriadas esqueléticas.'}, {'aid': 4, 'atex

In [ ]:
import spacy

class Vocabulary(object):
    def __init__(self, vocab2index={}, add_unk={}, unk_token='UNK'):
        self.vocab2index = vocab2index
        self.index2vocab = {idx:token for token, idx in self.vocab2index.items()}
        self.add_unk = add_unk
        self.unk_token = unk_token

    def lookup_token(self, token):
        if self.add_unk:
            return self.vocab2index.get(token, self.unk_index)
        else:
            return self.vocab2index[token]

    def lookup_index(self, index):
        if index not in self.index2vocab:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self.index2vocab[index]
    
    def __len__(self):
        return len(self.vocab2index)

    def add_token(self, token):
        if token in self.vocab2index:
            index = self.vocab2index[token]
        else:
            index = len(self.vocab2index)
            self.vocab2index[token] = index
            self.index2vocab[index] = token
        return index

class Vectorizer(object):
    def __init__(self, vocab, labels, vocab2index, label2index, tokenizer):
        self.sentence_vocab = vocab
        self.label_vocab = labels
        self.vocab2index = vocab2index
        self.label2index = label2index
        self.tokenizer = tokenizer

    def vectorize(self, s, N):
        padding_start = False
        x = self.tokenizer(s)
        enc = np.zeros(N, dtype=np.int32)
        enc1 = np.array([self.vocab2index.get(w, self.vocab2index["UNK"]) for w in x]) #value if w is in voca2index, else vocab2index["UNK"] 
        l = min(N, len(enc1))
        
        if padding_start:
            enc[:l] = enc1[:l]
        else:
            enc[N-l:] = enc1[:l]
        return enc

class HeadQA(Dataset):
    def __init__(self, data: dict, vectorizer, language='es'):
        self.data= load_dataset('head_qa', language)
        self.vectorizer = vectorizer


    def parse_dataset(training_data):
        for i in training_data:

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, x_size, n_classes): 
        super(LogisticRegression, self).__init__()             
        self.linear = nn.Linear(x_size, n_classes)
        
    def forward(self, x):
        return self.linear(x)